---
title: "Run simulations"
---

Subsidence calculations in `Atlans` consist of running a Simulation object that contains several parts: a Model, Clock, Forcings and time (start time and stress periods). Below an example is given of how to build and run the Simulation object with all the necessary components. Each Simulation stores an output Netcdf file with the calculated subsidence and individual contributions of consolidation, oxidation and shrinkage. We will start by importing the modules:

In [1]:
using Atlans
using Dates

# Processes to include in the subsidence calculation
The first choice a user has to make is which of the processes to include in the subsidence calculations. In part, an `Atlans` Model consists of a collection of SoilColumns that contain a subcolumn for groundwater, consolidation, oxidation and shrinkage (see [Model components](../core/index.qmd)). The processes leading to subsidence are determined at the voxel level. This means: each subcolumn contains the individual voxels with the specific process that will contribute to subsidence that is chosen by the user. 

Either of the processes, except for groundwater, can be included or excluded from the calculations. Below the options for consolidation, oxidation and shrinkage are given:

- Groundwater (mandatory)
  - HydrostaticGroundwater
- Consolidation
  - DrainingABCIsotache
  - ABCIsotache
  - Koppejan
  - NullConsolidation # this type is used to ignore consolidation
- Oxidation
  - CarbonStore
  - ConstantRate
  - NullOxidation # this type is used to ignore oxidation
- Shrinkage
  - SimpleShrinkage
  - NullShrinkage # this type is used to ignore shrinkage

# Build standard Model with all processes
We will build a Model component object that includes all of the processes. This is done using the voxelmodel and parameter table (see [homepage](../index.qmd)), and the chosen processes in the previous step. Additionally, a Model object contains AdaptiveCellsize and ExponentialTimeStepper objects. The AdaptiveCellsize object is an important component in the modelling as this controls splitting of voxels around the groundwater table. The ExponentialTimeStepper object is used to discretize the time steps within each stress period (see [API reference](../build/index.md)).

The code cell below shows the example to build a Model with all the processes included in the calculations.

In [ ]:
## specify the input voxelmodel and parameter table
path_to_subsurface_model = "my_subsurface_model.nc"
path_to_parameter_table = "my_parameter_table.nc"

## input for AdaptiveCellsize
max_voxel_thickness = 0.25 # m
split_tolerance = 0.01 # m

## input for Timestepper
start_day = 1.0
multiplier = 2

model = Atlans.Model(
    Atlans.HydrostaticGroundwater,
    Atlans.DrainingAbcIsotache,
    Atlans.CarbonStore,
    Atlans.OverConsolidationRatio,
    Atlans.SimpleShrinkage,
    Atlans.AdaptiveCellsize(max_voxel_thickness, split_tolerance),
    Atlans.ExponentialTimeStepper(start_day, multiplier),
    path_to_subsurface_model,
    path_to_parameter_table,
)

# Add forcings
`Atlans` simulations also need input for Forcings such as water level management (e.g. groundwater table lowering) at each stress period. The input for forcings is a namedtuple containing the names of the forcings as keys and the `Atlans` Forcing objects as values. See the [available]() for the available Forcing mechanisms and [API reference](../build/index.md) for the inputs. 

The example below shows how to include a single forcing (StageChange) in a Simulation:

In [ ]:
path_to_forcing = "stage_change.nc"

forcings = (
    stage_change=Atlans.StageChange(path_to_forcing)
)

# Stress periods
`Atlans` calculates subsidence over stress periods. A Simulation can be run over one or multiple stress periods. `Atlans` automatically determines the stress periods based on the time dimensions of the input forcings. An additional option is to define additional stess periods which are not in the time dimensions of the forcings, for instance to divide a single stress period in the forcing in smaller time steps.

For example: assume that the input StageChange has three stress periods of five years (2025, 2030 and 2035) but smaller time steps are desired in the first stress period. This can be done by providing a Vector containing DateTime objects of the additionally desired time steps:

In [ ]:
# Add yearly stress periods
additional_periods = map(
    DateTime,
    ['2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01']
)

# Build and run the Simulation
The only remaining input for the Simulation is the end date and a destination path of the output netcdf. The end date can be equal to the date of the last forcing period or correspond to an extra forcing period. The input for end date is a DateTime object. Now all the inputs are present and the Simulation object can be built: 

In [ ]:
output_path = "output.nc"
stop_time = DateTime("2040-01-01") # Chosen end date is an extra forcing period

simulation = Atlans.Simulation(
    model,
    output_path,
    stop_time,
    forcings,
    additional_periods
)

Finally, the Simulation can be run by:

In [ ]:
Atlans.run!(simulation)